In [1]:
import os
import librosa
import librosa.display
import numpy as np
import sys
import pickle

In [2]:
data_folder = './13_language_dataset/'
num_langs = len([name for name in os.listdir(data_folder) if not name.startswith('.')])

mel_spectrograms = './mel_spectrograms/'

In [9]:
l=0
data={}
labels={}
for lang in os.listdir(data_folder):
    f=1
    if not lang.startswith('.'):
        if not os.path.exists(mel_spectrograms+lang): #to create folders for each language
            os.makedirs(mel_spectrograms+lang)
        for wavfile in os.listdir(data_folder+lang):
            num_files = len([name for name in os.listdir(data_folder+lang) if not name.startswith('.')])
            sig, fs = librosa.load(data_folder+lang+'/'+wavfile,sr=None)
            
            a = 0.97
            emp_sig = np.append(sig[0],sig[1:] - a*sig[:-1]) #emphasized signal y(t)=x(t)-a*x(t-1)
            
            win_length = int(round(0.025*fs)) # for 25 ms frame length
            hop_length = int(round(0.01*fs)) # for 10ms shift between frames
            d = librosa.stft(emp_sig,n_fft=512,win_length=win_length,hop_length=hop_length)
            D = np.abs(d)**2 #to get power spectrum
            S = librosa.feature.melspectrogram(S=D)

            if (f%50) is 1: #To print status after every 50 generated spectrograms
                print('Generating spectrograms for {} ({}/{} wavfiles done). {} languages remaining'.format(lang,f,num_files,num_langs-l-1))
        
            if f is 1:
                data[l]=[]
                labels[l]=[]
            data[l].append(S)
            labels[l].append(l)

            f=f+1
        l=l+1

Generating spectrograms for manipuri (1/628 wavfiles done). 12 languages remaining
Generating spectrograms for manipuri (51/628 wavfiles done). 12 languages remaining
Generating spectrograms for manipuri (101/628 wavfiles done). 12 languages remaining
Generating spectrograms for manipuri (151/628 wavfiles done). 12 languages remaining
Generating spectrograms for manipuri (201/628 wavfiles done). 12 languages remaining
Generating spectrograms for manipuri (251/628 wavfiles done). 12 languages remaining
Generating spectrograms for manipuri (301/628 wavfiles done). 12 languages remaining
Generating spectrograms for manipuri (351/628 wavfiles done). 12 languages remaining
Generating spectrograms for manipuri (401/628 wavfiles done). 12 languages remaining
Generating spectrograms for manipuri (451/628 wavfiles done). 12 languages remaining
Generating spectrograms for manipuri (501/628 wavfiles done). 12 languages remaining
Generating spectrograms for manipuri (551/628 wavfiles done). 12 lan

Generating spectrograms for telugu (451/628 wavfiles done). 5 languages remaining
Generating spectrograms for telugu (501/628 wavfiles done). 5 languages remaining
Generating spectrograms for telugu (551/628 wavfiles done). 5 languages remaining
Generating spectrograms for telugu (601/628 wavfiles done). 5 languages remaining
Generating spectrograms for marathi (1/628 wavfiles done). 4 languages remaining
Generating spectrograms for marathi (51/628 wavfiles done). 4 languages remaining
Generating spectrograms for marathi (101/628 wavfiles done). 4 languages remaining
Generating spectrograms for marathi (151/628 wavfiles done). 4 languages remaining
Generating spectrograms for marathi (201/628 wavfiles done). 4 languages remaining
Generating spectrograms for marathi (251/628 wavfiles done). 4 languages remaining
Generating spectrograms for marathi (301/628 wavfiles done). 4 languages remaining
Generating spectrograms for marathi (351/628 wavfiles done). 4 languages remaining
Generating 

In [3]:
class_names = []
for lang in os.listdir(data_folder):
    if not lang.startswith('.'):
        class_names.append(lang)

class_names = np.asarray(class_names)
print('Languages to be classified and available in the dataset : {}'.format(class_names))

with open('class_names.pickle','wb') as f:
    pickle.dump(class_names,f)

Languages to be classified and available in the dataset : ['manipuri' 'urdu' 'bengali' 'kannada' 'odia' 'punjabi' 'tamil' 'telugu'
 'marathi' 'gujarati' 'malayalam' 'hindi' 'assamese']


In [12]:
for l in range(class_names.shape[0]):
    if l is 0:
        corpus = np.asarray(data[l])
        targets = np.asarray(labels[l])
    else:
        corpus = np.vstack((corpus,data[l]))
        targets = np.hstack((targets,labels[l]))

In [13]:
print('Dataset dimensions :')
print('Number of files = {}'.format(corpus.shape[0]))
print('Dimensions of each file = {}'.format(corpus[0].shape))
print('Targets = {}'.format(targets.shape[0]))

corpus = np.reshape(corpus,(corpus.shape[0],corpus.shape[1],corpus.shape[2],1))
print(corpus.shape)

with open('13L_mel_spectrograms.pickle','wb') as f:
    pickle.dump((corpus,targets),f)

Dataset dimensions :
Number of files = 8164
Dimensions of each file = (128, 501)
Targets = 8164
(8164, 128, 501, 1)
